In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
import glob
import json
import matplotlib.pyplot as plt
import math

In [2]:
all_csv_files = glob.glob("./Data/*.txt")
# print("--" + str(all_csv_files))
loops=len(all_csv_files)

In [3]:
data = {}
for idx,x in enumerate(all_csv_files):
    df = pd.read_csv(x,header=1,parse_dates=[[0,1]])
    df.columns = df.columns.str.replace(' ', '') 
    try:
        df.drop(df[df['Date_Time'] < pd.Timestamp('12/22/2020')].index, inplace = True)
    except TypeError:
        df.drop(df[df['Date_Time'] == '     0/0/0      0:0:0'].index, inplace = True)
        df['Date_Time'] = pd.to_datetime(df['Date_Time'])
        df.drop(df[df['Date_Time'] < pd.Timestamp('12/22/2020')].index, inplace = True)
    data[x[7:len(x)-4]] = df.reset_index(drop=True)

./Data\S-11.txt
./Data\S-12.txt
./Data\S-13.txt
./Data\S-14.txt
./Data\S-15.txt
./Data\S-4.txt
./Data\S-8.txt
./Data\S-9.txt


In [51]:
data['S-5']['Date_Time'][0]

Timestamp('2020-12-22 13:40:14')

In [32]:
df['Date_Time']

393     2020/12/22     14:2:0
394     2020/12/22    14:2:10
395     2020/12/22    14:2:20
396     2020/12/22    14:2:30
397     2020/12/22    14:2:40
                ...          
1510    2020/12/22     17:9:3
1511    2020/12/22    17:9:13
1512    2020/12/22    17:9:23
1513    2020/12/22    17:9:33
1514    2020/12/22    17:9:43
Name: Date_Time, Length: 1122, dtype: object